# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы «Смарт» или «Ультра». Необходимо построить модель для задачи классификации, которая выберет подходящий тариф.

Построим модель с максимально большим значением *accuracy*. Проверим *accuracy* на тестовой выборке.

## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Загрузим таблицу с данными
df = pd.read_csv('/datasets/users_behavior.csv')

# Изучим полученную таблицу
display(df.info())
display(df.shape)
display(df.head())
display(df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

(3214, 5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


**ВЫВОД**

В таблице 3214 строк, 5 столбцов, предобработка не требуется, т.к. выполнена ранее.

## Разобъем данные на выборки

In [2]:
# Разобьём данные на выборки
df_train_1, df_valid = train_test_split(df, test_size=0.2, random_state=12345)
df_train, df_test = train_test_split(df_train_1, test_size=0.25, random_state=12345)

# Проверим, правильно ли распределились данные
display(df_train.shape)
display(df_valid.shape)
display(df_test.shape)

display (len(df_train) / (len(df_train) + len(df_valid) + len(df_test)))
display (len(df_valid) / (len(df_train) + len(df_valid) + len(df_test)))
display (len(df_test) / (len(df_train) + len(df_valid) + len(df_test)))

(1928, 5)

(643, 5)

(643, 5)

0.5998755444928439

0.2000622277535781

0.2000622277535781

**ВЫВОД**

Мы видим, что данные распределены так, как мы и планировали: в обучающую выборку попало 60% данных, в валидационную и тестовую - по 20%. 

## Исследуем модели

In [3]:
# Сформируем параметры для обучения и проверки моделей
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

best_result = 0
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    # Обучим модель на тренировочной выборке
    model.fit(features_train, target_train)
    # Найдем предсказания на валидационной выборке
    predictions_valid = model.predict(features_valid) 
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))
    # Посчитаем качество модели на валидационной выборке
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        # Сохраним наилучшее значение метрики accuracy на валидационных данных
        best_result = result 
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

max_depth = 1 : 0.7480559875583204
max_depth = 2 : 0.7838258164852255
max_depth = 3 : 0.7869362363919129
max_depth = 4 : 0.7869362363919129
max_depth = 5 : 0.7884914463452566
max_depth = 6 : 0.7791601866251944
max_depth = 7 : 0.7884914463452566
max_depth = 8 : 0.7807153965785381
max_depth = 9 : 0.7776049766718507
max_depth = 10 : 0.7713841368584758
max_depth = 11 : 0.7651632970451011
max_depth = 12 : 0.7667185069984448
max_depth = 13 : 0.749611197511664
max_depth = 14 : 0.7542768273716952
max_depth = 15 : 0.7418351477449455
max_depth = 16 : 0.7433903576982893
max_depth = 17 : 0.744945567651633
max_depth = 18 : 0.7433903576982893
max_depth = 19 : 0.7278382581648523
Accuracy наилучшей модели на валидационной выборке: 0.7884914463452566


In [4]:
best_result = 0
for est in range(1, 20):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    # Обучим модель на тренировочной выборке
    model.fit(features_train, target_train) 
    # Найдем предсказания на валидационной выборке
    predictions_valid = model.predict(features_valid) 
    print("est =", est, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))
    # Посчитаем качество модели на валидационной выборке
    result = model.score(features_valid, target_valid) # посчитайте качество модели на валидационной выборке
    if result > best_result:
        # Сохраним наилучшее значение метрики accuracy на валидационных данных
        best_result = result
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

est = 1 : 0.7340590979782271
est = 2 : 0.7542768273716952
est = 3 : 0.7729393468118196
est = 4 : 0.7667185069984448
est = 5 : 0.7807153965785381
est = 6 : 0.7744945567651633
est = 7 : 0.7729393468118196
est = 8 : 0.776049766718507
est = 9 : 0.7791601866251944
est = 10 : 0.7869362363919129
est = 11 : 0.7822706065318819
est = 12 : 0.7822706065318819
est = 13 : 0.7916018662519441
est = 14 : 0.7884914463452566
est = 15 : 0.7900466562986003
est = 16 : 0.7838258164852255
est = 17 : 0.7838258164852255
est = 18 : 0.7853810264385692
est = 19 : 0.7838258164852255
Accuracy наилучшей модели на валидационной выборке: 0.7916018662519441


**ВЫВОДЫ**

Мы определили гиперпараметры, которые показывают наибольшее качество моделей.

## Проверим модель на тестовой выборке

In [5]:
model = DecisionTreeClassifier(random_state=12345, max_depth=5)
# Обучим модель на тренировочной выборке
model.fit(features_train, target_train)
# Найдем предсказания на тестовой выборке
predictions_test = model.predict(features_test) 
print(accuracy_score(target_test, predictions_test))

model = RandomForestClassifier(random_state=12345, n_estimators=13) # обучите модель с заданным количеством деревьев
 # Обучим модель на тренировочной выборке
model.fit(features_train, target_train) # обучите модель на тренировочной выборке
# Найдем предсказания на тестовой выборке
predictions_test = model.predict(features_test) 
print(accuracy_score(target_test, predictions_test))

0.7589424572317263
0.7776049766718507


**ВЫВОДЫ**

Значением accuracy при проверке моделей на тестовой выборке несколько ниже, чем на валидационной, однако выше 75%.

## Проверим модели на адекватность

In [6]:
# Сформируем столбец random_ultra из случайного набора 0 и 1
from random import randint

def random_ultra (row):
    row = randint(0, 1)
    return row
    
df['random_ultra'] = df.apply(random_ultra, axis=1)
display(df.head(15))
display(df.tail(15))

,calls,minutes,messages,mb_used,is_ultra,random_ultra
0,40.0,311.90,83.0,19915.42,0,1
1,85.0,516.75,56.0,22696.96,0,0
2,77.0,467.66,86.0,21060.45,0,0
3,106.0,745.53,81.0,8437.39,1,1
4,66.0,418.74,1.0,14502.75,0,1
5,58.0,344.56,21.0,15823.37,0,1
6,57.0,431.64,20.0,3738.90,1,1
7,15.0,132.40,6.0,21911.60,0,1
8,7.0,43.39,3.0,2538.67,1,1
9,90.0,665.41,38.0,17358.61,0,0


,calls,minutes,messages,mb_used,is_ultra,random_ultra
3199,104.0,702.47,0.0,23611.32,0,0
3200,0.0,0.00,55.0,10858.90,1,1
3201,56.0,419.42,59.0,5177.62,1,1
3202,19.0,163.10,10.0,17999.50,0,1
3203,53.0,390.39,85.0,30550.30,1,0
3204,86.0,658.66,47.0,14153.10,0,1
3205,59.0,412.81,16.0,14105.03,0,1
3206,76.0,586.51,54.0,14345.74,0,1
3207,17.0,92.39,2.0,4299.25,0,1
3208,164.0,1016.98,71.0,17787.52,1,0


In [7]:
# Разобьём обновленную таблицу на выборки
df_train_1, df_valid = train_test_split(df, test_size=0.2, random_state=12345)
df_train, df_test = train_test_split(df_train_1, test_size=0.25, random_state=12345)

# Сформируем параметры для обучения и проверки моделей
features_train_random = df_train.drop('random_ultra', axis=1)
target_train_random = df_train['random_ultra']
features_test_random = df_test.drop('random_ultra', axis=1)
target_test_random = df_test['random_ultra']

model = DecisionTreeClassifier(random_state=12345, max_depth=5)
# Обучим модель на тренировочной выборке
model.fit(features_train_random, target_train_random)
# Найдем предсказания на тестовой выборке
predictions_test = model.predict(features_test_random) 
print(accuracy_score(target_test_random, predictions_test))

model = RandomForestClassifier(random_state=12345, n_estimators=13) 
# Обучим модель на тренировочной выборке
model.fit(features_train_random, target_train_random) 
# Найдем предсказания на тестовой выборке
predictions_test = model.predict(features_test_random) 
print(accuracy_score(target_test_random, predictions_test))

0.49922239502332816
0.49144634525660963


**ВЫВОД**

Мы видим, что значение accuracy для моделей, обученных на случайных целевых значениях, существенно ниже, чем обученных на достоверных данных (около 50%), что говорит об адекватности наших моделей.